In [208]:
from bs4 import BeautifulSoup
import requests
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time
import numpy as np
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from datetime import date
from statistics import mean, median
from time import sleep
from datetime import datetime, timedelta
from selenium.webdriver.common.action_chains import ActionChains
from tqdm import tqdm

import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
import scipy.stats as stats
from statsmodels.miscmodels.ordinal_model import OrderedModel

# Parsing Data

In [411]:
df = pd.DataFrame({'Артикул':[], 'Цена': [], 'Название компании':[], 'Название товара': [], 'Количество отзывов': [], 
                   'Цвет': [],  'Рейтинг продавца':[], 'Рейтинг товара': [], 'Купили': [], '5 звезд': [], '4 звезды': [], '3 звезды': []
                  , '2 звезды': [], '1 звезда': []})



In [412]:
url = 'https://www.wildberries.ru/catalog/141734116/detail.aspx'

In [413]:
driver = webdriver.Chrome(r'/Users/konstantin_sukhin/Desktop/WB/chromedriver')
driver.get(url)


<ipython-input-413-8eb49c28eea1>:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r'/Users/konstantin_sukhin/Desktop/WB/chromedriver')


In [372]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
urls = soup.find(attrs={"class":{"catalog-page__main new-size"}}).find_all(attrs={"class":{"product-card__link j-card-link j-open-full-product-card"}})
len(urls)

100

In [391]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
urls += soup.find(attrs={"class":{"catalog-page__main new-size"}}).find_all(attrs={"class":{"product-card__link j-card-link j-open-full-product-card"}})
len(urls)

2000

In [426]:
for url in tqdm(urls):
    url = url.get('href')
    driver.get(url)
    sleep(2.5)
    lenOfPage = driver.execute_script("window.scrollTo(0, document.body.scrollHeight-1300);var lenOfPage=document.body.scrollHeight-1300;return lenOfPage;")
    match=False
    while(match==False):
        lastCount = lenOfPage
        sleep(1.3)
        lenOfPage = driver.execute_script("window.scrollTo(0, document.body.scrollHeight-1300);var lenOfPage=document.body.scrollHeight-1300;return lenOfPage;")
        if lastCount==lenOfPage:
            match=True
    sleep(1.5)
    try:
        soup=BeautifulSoup(driver.page_source, 'html.parser')
        if soup.find(attrs={"class":{"user-opinion__text"}}) != None:


            element = driver.find_element(By.CLASS_NAME,  'user-opinion__text' )


            action = ActionChains(driver)
            action.move_to_element(to_element=element).perform()
            scroll_vаlue = 10
            scroll_by = f'window.scrollBy(0, {scroll_vаlue});'
            driver.execute_script(scroll_by)  
            action.move_to_element(to_element=element).perform()
            sleep(1.5)
            soup=BeautifulSoup(driver.page_source, 'html.parser')
            article = re.findall(r'\d+', soup.find(attrs={"class":{"product-article"}}).text)[0]
            if soup.find(attrs={"class":{"price-block__final-price"}}) != None:
                coast = soup.find(attrs={"class":{"price-block__final-price"}}).text.strip().replace('\xa0', '').replace('₽', '')
            seller_name = soup.find(attrs={"class":{"seller-info__title"}}).text.strip()
            product_name = soup.find(attrs={"class":{"product-page__header"}}).span.text + ' / ' +soup.find(attrs={"class":{"product-page__header"}}).h1.text
            count_feedbacs = re.findall(r'\d+', soup.find(attrs={"class":{"product-review__count-review"}}).text.replace(' ', ''))[0]
            try:
                color = soup.find(attrs={"class":{"color"}}).text
            except:
                color = ''
            if soup.find(attrs={"class":{"address-rate-mini"}})!=None:
                rating_saller = soup.find(attrs={"class":{"address-rate-mini"}}).text
            else:
                rating_saller = 0
            rating_good = soup.find(attrs={"class":{"user-opinion__rating-numb"}}).text
            if soup.find(attrs={"class":{"product-order-quantity j-orders-count-wrapper"}}).text != '':
                count_cell = re.findall(r'\d+', soup.find(attrs={"class":{"product-order-quantity j-orders-count-wrapper"}}).text.replace('\xa0', ''))[0]
            else:
                count_cell = 0
            st5 = soup.find_all(attrs={"class":{"feedback-percent__count"}})[0].text
            st4 = soup.find_all(attrs={"class":{"feedback-percent__count"}})[1].text
            st3 = soup.find_all(attrs={"class":{"feedback-percent__count"}})[2].text
            st2 = soup.find_all(attrs={"class":{"feedback-percent__count"}})[3].text
            st1 = soup.find_all(attrs={"class":{"feedback-percent__count"}})[4].text
            df = df.append({'Артикул':article, 'Цена': coast, 'Название компании':seller_name, 'Название товара': product_name, 'Количество отзывов': count_feedbacs, 
                               'Цвет': color,  'Рейтинг продавца':rating_saller, 'Рейтинг товара': rating_good, 'Купили': count_cell, '5 звезд': st5, '4 звезды': st4, '3 звезды': st3
                              , '2 звезды': st2, '1 звезда': st1}, ignore_index=True)
        else:
            soup=BeautifulSoup(driver.page_source, 'html.parser')
            article = re.findall(r'\d+', soup.find(attrs={"class":{"product-article"}}).text)[0]
            if soup.find(attrs={"class":{"price-block__final-price"}}) != None:
                coast = soup.find(attrs={"class":{"price-block__final-price"}}).text.strip().replace('\xa0', '').replace('₽', '')
            seller_name = soup.find(attrs={"class":{"seller-info__title"}}).text.strip()
            product_name = soup.find(attrs={"class":{"product-page__header"}}).span.text + ' / ' +soup.find(attrs={"class":{"product-page__header"}}).h1.text
            count_feedbacs = re.findall(r'\d+', soup.find(attrs={"class":{"product-review__count-review"}}).text.replace(' ', ''))[0]
            try:
                color = soup.find(attrs={"class":{"color"}}).text
            except:
                color = ''
            if soup.find(attrs={"class":{"address-rate-mini"}})!= None:
                rating_saller = soup.find(attrs={"class":{"address-rate-mini"}}).text
            else:
                rating_saller = 0
            try:
                rating_good = soup.find(attrs={"class":{"user-opinion__rating-numb"}}).text
            except:
                rating_good = ''
            if soup.find(attrs={"class":{"product-order-quantity j-orders-count-wrapper"}}).text != '':
                count_cell = re.findall(r'\d+', soup.find(attrs={"class":{"product-order-quantity j-orders-count-wrapper"}}).text.replace('\xa0', ''))[0]
            else:
                count_cell = 0
            df = df.append({'Артикул':article, 'Цена': coast, 'Название компании':seller_name, 'Название товара': product_name, 'Количество отзывов': count_feedbacs, 
                               'Цвет': color,  'Рейтинг продавца':rating_saller, 'Рейтинг товара': rating_good, 'Купили': count_cell}, ignore_index=True)

    except:
        soup=BeautifulSoup(driver.page_source, 'html.parser')
        article = re.findall(r'\d+', soup.find(attrs={"class":{"product-article"}}).text)[0]
        if soup.find(attrs={"class":{"price-block__final-price"}}) != None:
            coast = soup.find(attrs={"class":{"price-block__final-price"}}).text.strip().replace('\xa0', '').replace('₽', '')
        seller_name = soup.find(attrs={"class":{"seller-info__title"}}).text.strip()
        product_name = soup.find(attrs={"class":{"product-page__header"}}).span.text + ' / ' +soup.find(attrs={"class":{"product-page__header"}}).h1.text
        count_feedbacs = re.findall(r'\d+', soup.find(attrs={"class":{"product-review__count-review"}}).text.replace(' ', ''))[0]
        try:
            color = soup.find(attrs={"class":{"color"}}).text
        except:
            color = ''
        if soup.find(attrs={"class":{"address-rate-mini"}})!=None:
            rating_saller = soup.find(attrs={"class":{"address-rate-mini"}}).text
        else:
            rating_saller = 0
        try:
            rating_good = soup.find(attrs={"class":{"user-opinion__rating-numb"}}).text
        except:
            rating_good = ''
        if soup.find(attrs={"class":{"product-order-quantity j-orders-count-wrapper"}}).text != '':
            count_cell = re.findall(r'\d+', soup.find(attrs={"class":{"product-order-quantity j-orders-count-wrapper"}}).text.replace('\xa0', ''))[0]
        else:
            count_cell = 0
        df = df.append({'Артикул':article, 'Цена': coast, 'Название компании':seller_name, 'Название товара': product_name, 'Количество отзывов': count_feedbacs, 
                           'Цвет': color,  'Рейтинг продавца':rating_saller, 'Рейтинг товара': rating_good, 'Купили': count_cell}, ignore_index=True)

100%|██████████| 721/721 [2:07:44<00:00, 10.63s/it]  


In [428]:
#df.to_excel("headphones4000.xlsx")

In [65]:
#df.to_excel("headphones2000.xlsx")

# Ordered Logit

In [2]:
df2000_st = pd.read_excel('headphones2000.xlsx')
df4000_st = pd.read_excel('headphones4000.xlsx')

df4000 = df2000_st.append(df4000_st).reset_index(drop=True)

In [3]:
def numbers(text):
    text = str(text)
    text = text.replace("\xa0", "")
    return int(re.findall(r'\d+', text)[0])

In [4]:
nona = df4000.copy()

In [5]:
nona['Количество отзывов'] = nona['Количество отзывов'].astype(float)
nona['Рейтинг продавца'] = nona['Рейтинг продавца'].astype(float)
nona['Цена'] = nona['Цена'].apply(numbers)
nona.loc[(nona['5 звезд'].isna()) & (nona['Количество отзывов']==0),'5 звезд'] = 0
nona.loc[(nona['4 звезды'].isna()) & (nona['Количество отзывов']==0),'4 звезды'] = 0
nona.loc[(nona['3 звезды'].isna()) & (nona['Количество отзывов']==0),'3 звезды'] = 0
nona.loc[(nona['2 звезды'].isna()) & (nona['Количество отзывов']==0),'2 звезды'] = 0
nona.loc[(nona['1 звезда'].isna()) & (nona['Количество отзывов']==0),'1 звезда'] = 0
nona.loc[(nona['Рейтинг товара'].isna()) & (nona['Количество отзывов']==0),'Рейтинг товара'] = 0
nona['Цвет'] = nona['Цвет'].fillna('no_color')
nona = nona.dropna()
nona['5 звезд'] = nona['5 звезд'].apply(numbers)
nona['4 звезды'] = nona['4 звезды'].apply(numbers)
nona['3 звезды'] = nona['3 звезды'].apply(numbers)
nona['2 звезды'] = nona['2 звезды'].apply(numbers)
nona['1 звезда'] = nona['1 звезда'].apply(numbers)
nona['Купили'] = nona['Купили'].apply(numbers)
nona['Купили'] = nona['Купили'].astype(int)
nona.loc[(nona['Рейтинг товара']=='') & (nona['Количество отзывов']==0),'Рейтинг товара'] = 0
nona['Рейтинг товара'] = nona['Рейтинг товара'].astype(float)

In [314]:
nona_X = nona.copy().reset_index(drop=True)
nona_X['tt'] = nona_X['Купили']-nona_X['Количество отзывов']
nona_X.loc[nona_X['tt']<0,'tt'] = np.NaN
nona_X = nona_X.dropna()

nona_X.loc[nona_X['tt']<20,'target']=1
nona_X.loc[(nona_X['tt']>=20) & (nona_X['tt']<100),'target']=2
nona_X.loc[(nona_X['tt']>=100) & (nona_X['tt']<500),'target']=3
nona_X.loc[(nona_X['tt']>=500) & (nona_X['tt']<2000),'target']=4
nona_X.loc[(nona_X['tt']>=2000),'target']=5

nona_X['index'] = nona_X['Название компании'].astype(str) + '_' + nona_X['Название товара'].astype(str) + '_' + nona_X['Цена'].astype(str)
nona_X.drop_duplicates(subset='index',inplace=True)
nona_X = nona_X.reset_index(drop=True)

In [318]:
dd = (nona_X['target'].value_counts()/nona_X['target'].value_counts().sum()).reset_index()
dd.rename(columns={'index':'Категория','target':'Доля наблюдений в категории'})

,Категория,Доля наблюдений в категории
0,1.0,0.543157
1,2.0,0.191680
2,3.0,0.102255
3,4.0,0.085148
4,5.0,0.077760


In [309]:
headphones = nona_X.copy()
headphones = headphones.drop(columns=['Название компании','Название товара',
                                      'Цвет','Рейтинг товара',
                                      'Купили','Артикул',
                                      '1 звезда',
                                      'Unnamed: 0',
                                     'index','tt'])
headphones = headphones.reset_index(drop=True)
headphones.head()

,Цена,Количество отзывов,Рейтинг продавца,5 звезд,4 звезды,3 звезды,2 звезды,target
0,527,32767.0,4.4,83,5,3,2,4.0
1,700,27311.0,4.5,84,6,3,2,4.0
2,1050,4688.0,4.8,93,3,1,1,4.0
3,587,6477.0,4.6,86,4,3,2,4.0
4,990,32767.0,4.8,92,3,2,1,4.0


In [310]:
#headphones[headphones.columns[:-1]].describe().T.to_excel('desc_reviews.xlsx')
desc = pd.read_excel('desc_reviews.xlsx')
desc = desc.rename(columns={'Unnamed: 0':'Признаки'})
desc['Среднее значение'] = desc['Среднее значение'].round(2)
desc['Стандартное отклонение'] = desc['Стандартное отклонение'].round(2)
desc

,Признаки,Среднее значение,Стандартное отклонение,Минимум,25% квантиль,50% квантиль,75% квантиль,Максимум
0,Цена,2012.92,3206.39,111,647.0,1159.0,2039.00,50243
1,Количество отзывов,357.31,2169.35,0,0.0,4.0,49.25,32767
2,Рейтинг продавца,4.26,0.42,0,4.1,4.3,4.50,5
3,5 звезд,48.75,39.44,0,0.0,64.0,83.00,100
4,4 звезды,4.72,11.34,0,0.0,0.0,6.00,100
5,3 звезды,3.32,9.50,0,0.0,0.0,4.00,100
6,2 звезды,3.32,11.57,0,0.0,0.0,2.00,100
7,1 звезда,11.05,21.31,0,0.0,3.0,11.00,100


In [311]:
X = headphones.drop(columns=['target'])
Y = headphones['target']

In [312]:
mod_prob = OrderedModel(Y,
                        X,distr='logit')

res_prob = mod_prob.fit(method='bfgs')
res_prob.summary()

Optimization terminated successfully.
         Current function value: 0.869915
         Iterations: 44
         Function evaluations: 51
         Gradient evaluations: 51


<class 'statsmodels.iolib.summary.Summary'>
"""
                             OrderedModel Results                             
==============================================================================
Dep. Variable:                 target   Log-Likelihood:                -2237.4
Model:                   OrderedModel   AIC:                             4489.
Method:            Maximum Likelihood   BIC:                             4530.
Date:                Wed, 10 May 2023                                         
Time:                        01:08:47                                         
No. Observations:                2572                                         
Df Residuals:                    2561                                         
Df Model:                           7                                         
======================================================================================
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
Цена               -4.555e-05   1.64e-05     -2.769      0.006   -7.78e-05   -1.33e-05
Количество отзывов     0.0056      0.000     17.740      0.000       0.005       0.006
Рейтинг продавца       0.8038      0.147      5.453      0.000       0.515       1.093
5 звезд                0.0335      0.002     20.261      0.000       0.030       0.037
4 звезды               0.0251      0.004      6.898      0.000       0.018       0.032
3 звезды               0.0314      0.004      7.329      0.000       0.023       0.040
2 звезды               0.0191      0.004      5.049      0.000       0.012       0.027
0.0/1.0                5.9673      0.629      9.493      0.000       4.735       7.199
1.0/2.0                0.3808      0.041      9.378      0.000       0.301       0.460
2.0/3.0                0.1787      0.058      3.066      0.002       0.064       0.293
3.0/4.0                0.7666      0.069     11.085      0.000       0.631       0.902
======================================================================================
"""

In [285]:
params = res_prob.params
params

Цена                 -0.000056
Количество отзывов    0.005398
Рейтинг продавца      0.611584
5 звезд               0.031084
4 звезды              0.020825
3 звезды              0.029397
2 звезды              0.016742
0.0/1.0               5.538807
1.0/2.0               0.165832
2.0/3.0              -0.279006
3.0/4.0               0.746961
dtype: float64